In [50]:
import pandas as pd
import pyomo.environ as pyo
from omlt import OmltBlock
from omlt.neuralnet import ReluBigMFormulation
from omlt.io import load_keras_sequential
import tensorflow
import pandas as pd
import pickle

In [51]:
dict_scalers = pickle.load(open("scalers.pkl", "rb"))
x_offset = dict_scalers["x_offset"]
x_factor = dict_scalers["x_factor"]
y_offset = dict_scalers["y_offset"]
y_factor = dict_scalers["y_factor"]
scaled_lb = dict_scalers["scaled_lb"]
scaled_ub = dict_scalers["scaled_ub"]
inputs = ["x", "y"]
outputs = ["f1", "f2"]

In [56]:
from omlt.neuralnet import FullSpaceSmoothNNFormulation
from omlt import OffsetScaling

scaler = OffsetScaling(
    offset_inputs={i: x_offset[inputs[i]] for i in range(len(inputs))},
    factor_inputs={i: x_factor[inputs[i]] for i in range(len(inputs))},
    offset_outputs={i: y_offset[outputs[i]] for i in range(len(outputs))},
    factor_outputs={i: y_factor[outputs[i]] for i in range(len(outputs))},
)

scaled_input_bounds = {i: (scaled_lb[i], scaled_ub[i]) for i in range(len(inputs))}

# Supposons que le modèle Keras soit déjà formé et défini comme 'model'

# 1. Charger le modèle Keras dans un format compatible avec OMLT
data = pd.read_parquet("../data/data.parquet")
X = data[['x', 'y']]
y = data[['f1', 'f2']]
nn = tensorflow.keras.models.load_model("plane_nn_relu.keras", compile=False)
net = load_keras_sequential(nn,
                            scaling_object=scaler, scaled_input_bounds=scaled_input_bounds)
#unscaled_input_bounds={0: (0, X['x'].max()), 1: (0, X['y'].max())})
# 2. Définir le modèle Pyomo pour l'optimisation
pyomo_model = pyo.ConcreteModel()

# Ajouter un bloc OMLT pour intégrer le réseau de neurones
pyomo_model.neural_network = OmltBlock()
#formulation = ReluBigMFormulation(net)
formulation = FullSpaceSmoothNNFormulation(net)
pyomo_model.neural_network.build_formulation(formulation)

f1 = pyomo_model.neural_network.outputs[0]
f2 = pyomo_model.neural_network.outputs[1]

# 3. Formuler l'objectif de maximisation de f1 - f2
pyomo_model.objective = pyo.Objective(expr=f2 - f1, sense=pyo.maximize)
pyomo_model.con_x = pyo.Constraint(expr=pyomo_model.neural_network.inputs[0] >= 0)
#pyomo_model.con_x_sup = pyo.Constraint(expr=pyomo_model.neural_network.inputs[0] <= 2)

pyomo_model.con_y = pyo.Constraint(expr=pyomo_model.neural_network.inputs[1] >= 0)

# 4. Résoudre le problème
solver = pyo.SolverFactory("ipopt")
status = solver.solve(pyomo_model, tee=False)

# 5. Afficher les résultats optimaux
optimal_x = pyo.value(pyomo_model.neural_network.inputs[0])
optimal_y = pyo.value(pyomo_model.neural_network.inputs[1])
optimal_f1 = pyo.value(f1)
optimal_f2 = pyo.value(f2)
optimal_value = optimal_f2 + optimal_f1

print(f'Valeurs optimales : x = {optimal_x}, y = {optimal_y}')
print(f'f1 = {optimal_f1}, f2 = {optimal_f2}')
print(f'Valeur de f1 - f2 maximale : {optimal_value}')


Valeurs optimales : x = 2.5649608761183784, y = 3.6816727828970146
f1 = 0.7804700618054868, f2 = 1.5834438569762697
Valeur de f1 - f2 maximale : 2.3639139187817566


In [72]:
x = optimal_x
y = optimal_y
x_scaled = (x - x_offset["x"])/(x_factor["x"])
y_scaled = (y - x_offset["y"])/(x_factor["y"])
res = nn.predict(pd.DataFrame([{"x": x_scaled, "y": y_scaled}]))
res[0][0] * y_factor["f1"] + y_offset["f1"]
res[0][1] * y_factor["f2"] + y_offset["f2"]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


1.5834438808520335

In [69]:
y_scaled

0.06005884521227968